# Text Classification

In [1]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from os import listdir
from os.path import isfile, join
import pandas as pd
import os
import re
from sklearn.decomposition import PCA

### Specifying directory

In [3]:
path = "C:\\Users\\Alienware\\OneDrive\\Desktop\\training" # This link is changeable 
dlinks = [x[0] for x in os.walk(path)]

### Reading directory content

In [4]:
def ReadFolderNames(dlinks):
    dnames = []
    for i in range(len(dlinks)):
        dnames.append(dlinks[i].replace(path + "\\", ''))
    dnames.pop(0)
    return dnames

In [5]:
def ReadFilePaths(dlinks):
    files = []
    for folder in dlinks:
        files.append([f for f in listdir(folder) if isfile(join(folder, f))])
    return files

### Creating a DataFrame including the content

In [6]:
folders = ReadFolderNames(dlinks)
print(len(folders))

paths = ReadFilePaths(dlinks)
paths.pop(0)
print(len(paths))

91
91


In [9]:
folders

['acq',
 'alum',
 'barley',
 'bop',
 'carcass',
 'castor-oil',
 'cocoa',
 'coconut',
 'coconut-oil',
 'coffee',
 'copper',
 'copra-cake',
 'corn',
 'cotton',
 'cotton-oil',
 'cpi',
 'cpu',
 'crude',
 'dfl',
 'dlr',
 'dmk',
 'earn',
 'fuel',
 'gas',
 'gnp',
 'gold',
 'grain',
 'groundnut',
 'groundnut-oil',
 'heat',
 'hog',
 'housing',
 'income',
 'instal-debt',
 'interest',
 'ipi',
 'iron-steel',
 'jet',
 'jobs',
 'l-cattle',
 'lead',
 'lei',
 'lin-oil',
 'livestock',
 'lumber',
 'meal-feed',
 'money-fx',
 'money-supply',
 'naphtha',
 'nat-gas',
 'nickel',
 'nkr',
 'nzdlr',
 'oat',
 'oilseed',
 'orange',
 'palladium',
 'palm-oil',
 'palmkernel',
 'pet-chem',
 'platinum',
 'potato',
 'propane',
 'rand',
 'rape-oil',
 'rapeseed',
 'reserves',
 'retail',
 'rice',
 'rubber',
 'rye',
 'ship',
 'silver',
 'sorghum',
 'soy-meal',
 'soy-oil',
 'soybean',
 'strategic-metal',
 'sugar',
 'sun-meal',
 'sun-oil',
 'sunseed',
 'tea',
 'tin',
 'trade',
 'unknown',
 'veg-oil',
 'wheat',
 'wpi',
 'yen'

In [8]:
paths

[['0000005',
  '0000007',
  '0000027',
  '0000028',
  '0000044',
  '0000065',
  '0000078',
  '0000087',
  '0000090',
  '0000095',
  '0000096',
  '0000112',
  '0000114',
  '0000117',
  '0000132',
  '0000133',
  '0000143',
  '0000185',
  '0000209',
  '0000212',
  '0000214',
  '0000224',
  '0000234',
  '0000236',
  '0000254',
  '0000255',
  '0000256',
  '0000258',
  '0000259',
  '0000260',
  '0000262',
  '0000265',
  '0000268',
  '0000269',
  '0000271',
  '0000275',
  '0000279',
  '0000281',
  '0000282',
  '0000289',
  '0000298',
  '0000304',
  '0000309',
  '0000313',
  '0000314',
  '0000315',
  '0000318',
  '0000319',
  '0000333',
  '0000336',
  '0000337',
  '0000339',
  '0000352',
  '0000353',
  '0000354',
  '0000357',
  '0000358',
  '0000378',
  '0000388',
  '0000390',
  '0000392',
  '0000393',
  '0000395',
  '0000399',
  '0000400',
  '0000406',
  '0000408',
  '0000428',
  '0000430',
  '0000433',
  '0000434',
  '0000436',
  '0000441',
  '0000442',
  '0000444',
  '0000446',
  '0000447',

### Splitting files

In [12]:
data = {'Folder Name': folders, 'Files': paths}
df = pd.DataFrame(data)
df['Text'] = ""

In [13]:
df = df.explode('Files').reset_index(drop=True)

In [14]:
df

,Folder Name,Files,Text
0,acq,0000005,
1,acq,0000007,
2,acq,0000027,
3,acq,0000028,
4,acq,0000044,
...,...,...,...
11408,zinc,0007379,
11409,zinc,0007855,
11410,zinc,0008905,
11411,zinc,0009423,


### Inserting files' texts into the DataFrame

In [15]:
for fno in range(len(df['Files'])):
    f = open(path + "\\" + str(df['Folder Name'][fno]) + "\\" + str(df['Files'][fno]), "rt")
    df['Text'][fno] = f.readlines()

In [16]:
df

,Folder Name,Files,Text
0,acq,0000005,"[\n, \n, COMPUTER TERMINAL SYSTEMS <CPML> COMP..."
1,acq,0000007,"[\n, \n, OHIO MATTRESS <OMT> MAY HAVE LOWER 1S..."
2,acq,0000027,"[\n, \n, MCLEAN'S <MII> U.S. LINES SETS ASSET ..."
3,acq,0000028,"[\n, \n, CHEMLAWN <CHEM> RISES ON HOPES FOR HI..."
4,acq,0000044,"[\n, \n, <COFAB INC> BUYS GULFEX FOR UNDISCLOS..."
...,...,...,...
11408,zinc,0007379,"[\n, \n, PEGASUS GOLD <PGULF> STARTS MILLING I..."
11409,zinc,0007855,"[\n, \n, WORLD ZINC STOCKS FALL 7,700 TONNES I..."
11410,zinc,0008905,"[\n, \n, LME DETAILS MARCH 1987 TURNOVER\n, \n..."
11411,zinc,0009423,"[\n, \n, BALL <BLL> TO SUPPLY PENNY BLANKS TO ..."


### Turning texts into strings (not lists)

In [17]:
for indx, file in df['Text'].iteritems():
    string = ' '.join(item for item in file)
    df['Text'][indx] = string

In [18]:
df['Text']

0        \n \n COMPUTER TERMINAL SYSTEMS <CPML> COMPLET...
1        \n \n OHIO MATTRESS <OMT> MAY HAVE LOWER 1ST Q...
2        \n \n MCLEAN'S <MII> U.S. LINES SETS ASSET TRA...
3        \n \n CHEMLAWN <CHEM> RISES ON HOPES FOR HIGHE...
4        \n \n <COFAB INC> BUYS GULFEX FOR UNDISCLOSED ...
                               ...                        
11408    \n \n PEGASUS GOLD <PGULF> STARTS MILLING IN M...
11409    \n \n WORLD ZINC STOCKS FALL 7,700 TONNES IN F...
11410    \n \n LME DETAILS MARCH 1987 TURNOVER\n \n    ...
11411    \n \n BALL <BLL> TO SUPPLY PENNY BLANKS TO MIN...
11412    \n \n WESTMIN TO RAISE MYRA FALLS CAPACITY BY ...
Name: Text, Length: 11413, dtype: object

### Appending words that include letters only

In [19]:
df["Splitted_Text"] = df["Text"].str.split(" ")
for indx, text in df['Splitted_Text'].iteritems():
    words = []
    for word in text:
        if re.match(r'^[a-zA-Z]', word):
            words.append(word)
    df['Splitted_Text'][indx] == words
    print(words[0])

COMPUTER
OHIO
MCLEAN'S
CHEMLAWN
INC>
INVESTMENT
AMERICAN
HONG
LIEBERT
GULF
INVESTMENT
DREXEL
WINS
SUFFIELD
VERSATILE
VIDEOTRON
CIRCUIT
NIPPON
SHV
WALLENBERGS
SHV
SALE
EXCO
COLOROLL
SCIENTIFIC
AMERICAN
ROPAK
PENRIL
BURDETT
PUROLATOR
FINANCIAL
MARRIOTT
VIACOM
LAROCHE
SENIOR
VIACOM
PITTSTON
DIAGNOSTIC/RETRIEVAL
MILLER
PITTSTON
CONSOLIDATED
DIAGNOSTIC
THE
CORNING
BANK
CORNING
BALLY
CONSOLIDATED
AMERICAN
MULTI-STEP
ESSELTE
FOUR
REDSTONE
MONTEDISON
UTILICORP
CARBIDE
CORRECTED
KAPOK
INVESTOR
BANNER
CORRECTED
OSR
CRONUS
AG>
AMERICUS
INC>
CORP>
CONVERGENT
ROSPATCH
ROSPATCH
NORTH
TALKING
WAVEHILL
SECURITY
ICI
ALBERTSON'S
ROSPATCH
INT'L
JWT
FOOTE
UTILICORP
PESCH
U.S.
AMCA
SCOTT'S
UNION
ANALYSTS
FERRO
CCL
ARMTEK
LEUCADIA
MICROBIO
PAINEWEBBER<PWJ>
REXNORD
CANTREX
INVESTMENT
SEC
TRANSAMERICA
U.K.
KLM
ULTRAMAR
WOOLWORTH,
HK
BRYSON
GREENWOOD
HEINZ
VIACOM
NATIONAL
SARA
SCOTT'S
VIDEO
COMMUNITY
CALMAR
ITALY'S
NORTON
SAATCHI
UNILEVER'S
GROUP
TYLAN
PANTERA
LIFETIME
BENEFICIAL
IMTEC
CORRECTED
TENDER
VARIAN


In [20]:
df['Splitted_Text']

0        [\n, \n, COMPUTER, TERMINAL, SYSTEMS, <CPML>, ...
1        [\n, \n, OHIO, MATTRESS, <OMT>, MAY, HAVE, LOW...
2        [\n, \n, MCLEAN'S, <MII>, U.S., LINES, SETS, A...
3        [\n, \n, CHEMLAWN, <CHEM>, RISES, ON, HOPES, F...
4        [\n, \n, <COFAB, INC>, BUYS, GULFEX, FOR, UNDI...
                               ...                        
11408    [\n, \n, PEGASUS, GOLD, <PGULF>, STARTS, MILLI...
11409    [\n, \n, WORLD, ZINC, STOCKS, FALL, 7,700, TON...
11410    [\n, \n, LME, DETAILS, MARCH, 1987, TURNOVER\n...
11411    [\n, \n, BALL, <BLL>, TO, SUPPLY, PENNY, BLANK...
11412    [\n, \n, WESTMIN, TO, RAISE, MYRA, FALLS, CAPA...
Name: Splitted_Text, Length: 11413, dtype: object

### Removing unnecessary suffixes added into words

In [21]:
for text in range(len(df['Splitted_Text'])):
    for word in range(len(df['Splitted_Text'][text])):
        df['Splitted_Text'][text][word] = re.sub("\n", "", df['Splitted_Text'][text][word])
        df['Splitted_Text'][text][word] = re.sub("\.", "", df['Splitted_Text'][text][word])
        df['Splitted_Text'][text][word] = re.sub(",", "", df['Splitted_Text'][text][word])

In [22]:
for text in range(len(df['Splitted_Text'])):
    delete_empty = [ele for ele in df['Splitted_Text'][text] if ele.strip()]
    df['Splitted_Text'][text] = delete_empty

In [23]:
df['Splitted_Text']

0        [COMPUTER, TERMINAL, SYSTEMS, <CPML>, COMPLETE...
1        [OHIO, MATTRESS, <OMT>, MAY, HAVE, LOWER, 1ST,...
2        [MCLEAN'S, <MII>, US, LINES, SETS, ASSET, TRAN...
3        [CHEMLAWN, <CHEM>, RISES, ON, HOPES, FOR, HIGH...
4        [<COFAB, INC>, BUYS, GULFEX, FOR, UNDISCLOSED,...
                               ...                        
11408    [PEGASUS, GOLD, <PGULF>, STARTS, MILLING, IN, ...
11409    [WORLD, ZINC, STOCKS, FALL, 7700, TONNES, IN, ...
11410    [LME, DETAILS, MARCH, 1987, TURNOVER, LONDON, ...
11411    [BALL, <BLL>, TO, SUPPLY, PENNY, BLANKS, TO, M...
11412    [WESTMIN, TO, RAISE, MYRA, FALLS, CAPACITY, BY...
Name: Splitted_Text, Length: 11413, dtype: object

### Stemming words

In [24]:
ps = PorterStemmer()
df['Stemmed_Text'] = None
for index, text in df['Splitted_Text'].iteritems():
    stemmed = []
    for word in text:
        stemmed.append(ps.stem(word))
    df['Stemmed_Text'][index] = stemmed

In [23]:
df['Stemmed_Text']

0        [comput, termin, system, <cpml>, complet, sale...
1        [ohio, mattress, <omt>, may, have, lower, 1st,...
2        [mclean', <mii>, us, line, set, asset, transfe...
3        [chemlawn, <chem>, rise, on, hope, for, higher...
4        [<cofab, inc>, buy, gulfex, for, undisclos, am...
                               ...                        
11408    [pegasu, gold, <pgulf>, start, mill, in, monta...
11409    [world, zinc, stock, fall, 7700, tonn, in, feb...
11410    [lme, detail, march, 1987, turnov, london, apr...
11411    [ball, <bll>, to, suppli, penni, blank, to, mi...
11412    [westmin, to, rais, myra, fall, capac, by, 33,...
Name: Stemmed_Text, Length: 11413, dtype: object

### Removing stop words

In [24]:
stop_words = set(stopwords.words('english'))
for index, lis in df['Stemmed_Text'].iteritems(): 
    filtered_words = []
    for w in lis:
        if w.lower() not in stop_words:
            filtered_words.append(w)
    df['Stemmed_Text'][index] = filtered_words

In [25]:
df['Stemmed_Text']

0        [comput, termin, system, <cpml>, complet, sale...
1        [ohio, mattress, <omt>, may, lower, 1st, qtr, ...
2        [mclean', <mii>, us, line, set, asset, transfe...
3        [chemlawn, <chem>, rise, hope, higher, bid, <a...
4        [<cofab, inc>, buy, gulfex, undisclos, amount,...
                               ...                        
11408    [pegasu, gold, <pgulf>, start, mill, montana, ...
11409    [world, zinc, stock, fall, 7700, tonn, februar...
11410    [lme, detail, march, 1987, turnov, london, apr...
11411    [ball, <bll>, suppli, penni, blank, mint, munc...
11412    [westmin, rais, myra, fall, capac, 33, pct, ca...
Name: Stemmed_Text, Length: 11413, dtype: object

### Overviewing Dataframe

In [25]:
df['joint_final']=df['Stemmed_Text'].apply(lambda x: ' '.join(x))

In [26]:
df

,Folder Name,Files,Text,Splitted_Text,Stemmed_Text,joint_final
0,acq,0000005,\n \n COMPUTER TERMINAL SYSTEMS <CPML> COMPLET...,"[COMPUTER, TERMINAL, SYSTEMS, <CPML>, COMPLETE...","[comput, termin, system, <cpml>, complet, sale...",comput termin system <cpml> complet sale comma...
1,acq,0000007,\n \n OHIO MATTRESS <OMT> MAY HAVE LOWER 1ST Q...,"[OHIO, MATTRESS, <OMT>, MAY, HAVE, LOWER, 1ST,...","[ohio, mattress, <omt>, may, have, lower, 1st,...",ohio mattress <omt> may have lower 1st qtr net...
2,acq,0000027,\n \n MCLEAN'S <MII> U.S. LINES SETS ASSET TRA...,"[MCLEAN'S, <MII>, US, LINES, SETS, ASSET, TRAN...","[mclean', <mii>, US, line, set, asset, transfe...",mclean' <mii> US line set asset transfer cranf...
3,acq,0000028,\n \n CHEMLAWN <CHEM> RISES ON HOPES FOR HIGHE...,"[CHEMLAWN, <CHEM>, RISES, ON, HOPES, FOR, HIGH...","[chemlawn, <chem>, rise, ON, hope, for, higher...",chemlawn <chem> rise ON hope for higher bid <a...
4,acq,0000044,\n \n <COFAB INC> BUYS GULFEX FOR UNDISCLOSED ...,"[<COFAB, INC>, BUYS, GULFEX, FOR, UNDISCLOSED,...","[<cofab, inc>, buy, gulfex, for, undisclos, am...",<cofab inc> buy gulfex for undisclos amount ho...
...,...,...,...,...,...,...
11408,zinc,0007379,\n \n PEGASUS GOLD <PGULF> STARTS MILLING IN M...,"[PEGASUS, GOLD, <PGULF>, STARTS, MILLING, IN, ...","[pegasu, gold, <pgulf>, start, mill, IN, monta...",pegasu gold <pgulf> start mill IN montana jeff...
11409,zinc,0007855,"\n \n WORLD ZINC STOCKS FALL 7,700 TONNES IN F...","[WORLD, ZINC, STOCKS, FALL, 7700, TONNES, IN, ...","[world, zinc, stock, fall, 7700, tonn, IN, feb...",world zinc stock fall 7700 tonn IN februari ei...
11410,zinc,0008905,\n \n LME DETAILS MARCH 1987 TURNOVER\n \n ...,"[LME, DETAILS, MARCH, 1987, TURNOVER, LONDON, ...","[lme, detail, march, 1987, turnov, london, apr...",lme detail march 1987 turnov london april 2 - ...
11411,zinc,0009423,\n \n BALL <BLL> TO SUPPLY PENNY BLANKS TO MIN...,"[BALL, <BLL>, TO, SUPPLY, PENNY, BLANKS, TO, M...","[ball, <bll>, TO, suppli, penni, blank, TO, mi...",ball <bll> TO suppli penni blank TO mint munci...


### Performing TF-IDF on stemmed words, in addition to tokenizing them

In [27]:
tfidf = TfidfVectorizer()
vec = tfidf.fit_transform(df['joint_final'])

In [ ]:
tfidf

In [ ]:
vec

In [28]:
ndf = pd.DataFrame(vec.toarray(), columns = tfidf.get_feature_names()) 

In [29]:
ndf

,000,0001,0006913,0006916,0007050,0007100,0007150,001,0015,0020,...,zuccherifici,zuckerman,zuercher,zulia,zur,zurich,zuyuan,zverev,zy,zzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Performing Diemnstionality Reduction using PCA


In [ ]:
pca = PCA()
pca_method = pca.fit_transform(ndf)

In [ ]:
pca_method

In [ ]:
ndf = pd.DataFrame(pca_method)

In [41]:
ndf

,0,1,2,3,4,5,6,7,8,9,...,11403,11404,11405,11406,11407,11408,11409,11410,11411,11412
0,-0.060297,-0.071363,0.055952,-0.026602,-0.141487,-0.039091,0.055301,-0.057685,0.034852,-0.028705,...,3.305633e-20,5.682291e-21,-2.292518e-22,-4.857184e-21,6.229325e-22,2.146191e-21,4.102212e-23,-1.819735e-22,5.168609e-22,9.053149e-23
1,0.012452,-0.071010,-0.003287,-0.026310,-0.104608,-0.012741,-0.051016,-0.092150,0.022549,0.011061,...,-2.309635e-18,-1.305450e-18,6.062553e-19,3.922076e-19,-3.996555e-19,-5.741326e-19,2.580669e-19,-1.493194e-19,-6.724544e-19,-1.783421e-20
2,-0.067009,-0.049680,0.076272,0.032721,-0.037288,-0.081033,0.066461,-0.022209,-0.009370,-0.028621,...,5.253137e-19,2.225592e-19,3.580852e-22,-2.626492e-19,7.530763e-20,-8.006741e-20,1.240724e-20,-1.347610e-19,1.068751e-19,-2.256016e-20
3,-0.062063,-0.057227,0.034575,-0.024424,-0.083434,-0.050183,0.051621,-0.031469,0.008658,-0.007550,...,3.305633e-20,5.682291e-21,-2.292518e-22,-4.857184e-21,6.229325e-22,2.146191e-21,4.102212e-23,-1.819735e-22,5.168609e-22,9.053149e-23
4,-0.048225,-0.034134,0.075716,0.006730,-0.040264,-0.031890,0.044725,-0.035323,0.038535,-0.037498,...,3.305633e-20,5.682291e-21,-2.292518e-22,-4.857184e-21,6.229325e-22,2.146191e-21,4.102212e-23,-1.819735e-22,5.168609e-22,9.053149e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11408,-0.034161,-0.052871,0.019507,0.000337,-0.088540,-0.005551,0.038418,-0.050247,0.021741,-0.034683,...,-3.598088e-17,1.194261e-17,-3.495320e-18,-1.829049e-17,-7.621107e-18,-3.303770e-17,1.291709e-18,6.496893e-18,9.755524e-19,1.638518e-18
11409,-0.070372,-0.049282,-0.011590,0.157122,0.011355,0.146202,-0.116212,-0.063237,0.075152,-0.000232,...,3.305633e-20,5.682291e-21,-2.292518e-22,-4.857184e-21,6.229325e-22,2.146191e-21,4.102212e-23,-1.819735e-22,5.168609e-22,9.053149e-23
11410,-0.041924,-0.032524,0.058721,0.027885,-0.025490,0.029542,0.016516,0.022940,0.041490,-0.019546,...,2.199611e-18,-1.301457e-17,-2.095077e-17,-5.017954e-18,-2.319353e-17,2.628224e-18,-9.416822e-18,-5.455983e-18,6.922892e-18,4.523727e-18
11411,-0.051119,-0.033499,0.059857,0.017410,-0.010719,-0.017861,0.023401,-0.006306,0.022637,-0.014852,...,-5.225634e-17,-2.623823e-17,2.615130e-17,1.574542e-18,8.057533e-18,1.919801e-17,3.462554e-18,-6.193755e-18,7.582602e-18,8.111967e-19


### Encoding the data and labeling it

In [36]:
labelled = pd.DataFrame()
labelled['label'] = df['Folder Name']

In [37]:
labelencoder = preprocessing.LabelEncoder()
labelled['encoded_label'] = labelencoder.fit_transform(labelled['label'])
labelled

,label,encoded_label
0,acq,0
1,acq,0
2,acq,0
3,acq,0
4,acq,0
...,...,...
11408,zinc,90
11409,zinc,90
11410,zinc,90
11411,zinc,90


## Algorithms

In [38]:
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

### Splitting data

In [42]:
x_train, x_test, y_train, y_test = train_test_split(ndf, labelled['encoded_label'],test_size=0.20)

### SVM

In [43]:
SVM = svm.SVC(decision_function_shape='ovo')
SVM.fit(x_train, y_train)
SVM.score(x_test,y_test)

0.6727989487516426

### Adaboost

In [44]:
adaboost = AdaBoostClassifier(n_estimators=80)
adaboost.fit(x_train, y_train)
adaboost.score(x_test, y_test)

0.35435830048182215

### Random Forest

In [45]:
random_forest = RandomForestClassifier()
random_forest.fit(x_train, y_train)
random_forest.score(x_test, y_test)

0.4879544459045116

In [46]:
SVM_prediction=SVM.predict(x_test)
adaboost_prediction=adaboost.predict(x_test)
random_forest_prediction=random_forest.predict(x_test)

In [54]:
y_test

4786     21
2441     17
11150    87
240       0
9878     85
         ..
1556      0
5842     24
6019     26
1528      0
9053     84
Name: encoded_label, Length: 2283, dtype: int32

### Testing the results

In [55]:
print("Random Forest algorithm classification prediction sample 1 :",random_forest_prediction[0])
print("Adaboost algorithm classification prediction sample 1 :",adaboost_prediction[0])
print("SVM algorithm classification prediction sample 1 :",SVM_prediction[0])

Random Forest algorithm classification prediction sample 1 : 21
Adaboost algorithm classification prediction sample 1 : 21
SVM algorithm classification prediction sample 1 : 21


In [56]:
print("Actual Result for sample 1: ",y_test[4786])

Actual Result for sample 1:  21
